In [38]:
from stardict import open_dict
import pandas as pd

df = pd.read_csv("Collins_1.csv", header=None)
df.columns = ["word"]
df["collins"] = 5
df["phonetic"] = ''
df["definition"] = ''
df["translation"] = ''
df["exchange"] = ''
d = open_dict("stardict.db")
print(d.query("agree"))

for i in range(len(df)):
    row = df.iloc[i]
    dic = d.query(row["word"])
    if dic is not None:
        df.at[i, "collins"] = dic["collins"]
        df.at[i, "phonetic"] = dic['phonetic'].replace("\r\n", ";").replace("\n", ";")
        df.at[i, "definition"] = dic['definition'].replace("\r\n", ";").replace("\n", ";")
        df.at[i, "translation"] = dic['translation'].replace("\r\n", ";").replace("\n", ";")
        df.at[i, "exchange"] = dic['exchange'].replace("\r\n", ";").replace("\n", ";")

df.head()

{'id': 75432, 'word': 'agree', 'sw': 'agree', 'phonetic': "ә'gri:", 'definition': 'v. be in accord; be in agreement\nv. consent or assent to a condition, or agree to do something\nv. show grammatical agreement\nv. be agreeable or suitable', 'translation': 'vi. 同意, 赞成, 应允, 适合\nvt. 承认, 认定, 同意', 'pos': 'v:100', 'collins': 5, 'oxford': 1, 'tag': 'zk gk', 'bnc': 401, 'frq': 514, 'exchange': 'd:agreed/p:agreed/3:agrees/i:agreeing', 'detail': None, 'audio': ''}


,word,collins,phonetic,definition,translation,exchange
0,aback,1.0,ә'bæk,r. having the wind against the forward side of...,"adv. 向后, 朝后, 突然, 船顶风地",
1,abate,1.0,ә'beit,v. become less in amount or intensity,"vt. 减少, 减轻, 减弱, 废除, 打折扣;vi. 减轻, 减弱, 减少, 失效, 被废除",d:abated/i:abating/p:abated/3:abates/s:abates
2,abdomen,1.0,'æbdәmen,n. the region of the body of a vertebrate betw...,n. 腹部;[医] 腹[部],s:abdomens
3,abduct,1.0,æb'dʌkt,v. pull away from the body,"vt. 诱拐, 绑架, 使外展;[医] 外展, 展",d:abducted/p:abducted/i:abducting/3:abducts/s:...
4,aberration,1.0,æbә'reiʃәn,n. a disorder in one's mental state;n. an opti...,"n. 离开正路, 偏离, 畸变, 光行差, 心理失常, 色差;[化] 光行差; 像差",s:aberrations


In [39]:
df.to_csv("Collins_1_query.csv", index=False)

In [21]:
print("v. be in accord; be in agreement\nv. consent or assent to a condition, or agree to do something\nv. show grammatical agreement\nv. be agreeable or suitable".replace("\n", ";"))

v. be in accord; be in agreement;v. consent or assent to a condition, or agree to do something;v. show grammatical agreement;v. be agreeable or suitable


# generate anki flash cards

In [47]:
import pandas as pd

df = pd.read_csv("Collins_1_query.csv")
df = df.fillna("")

In [42]:
def convert_str(s):
    s = s.replace("<", "[").replace(">", "]")
    tokens = s.split(";")
    if len(tokens) > 1:
        result = []
        for token in tokens:
            result.append(f"<div>{token}</div>")
        return "".join(result)

    return s


def row_to_note(r):
    name = r["word"]
    answer = f"""
<h3>phonetic</h3><div>{r["phonetic"]}</div>
<h3>definition</h3><div>{convert_str(str(r["definition"]))}</div>
<h3>translation</h3><div>{convert_str(str(r["translation"]))}</div>
<h3>exchange</h3><div>{r["exchange"]}</div>
    """
    return [name, answer]

In [32]:
r = row_to_note(df.iloc[100])
print(r[0])
print(r[1])

chief

<h3>phonetic</h3><div>tʃi:f</div>
<h3>definition</h3><div>s. most important element</div>
<h3>translation</h3><div><div>n. 领袖, 酋长, 长官, 主要部分</div><div>a. 主要的, 首位的</div></div>
<h3>exchange</h3><div>s:chiefs</div>
    


In [48]:
import random
import genanki

deck_id = random.randrange(1 << 30, 1 << 31)
model_id = random.randrange(1 << 30, 1 << 31)

my_deck = genanki.Deck(deck_id, "Collins 1")
my_model = genanki.Model(
    model_id,
    'Simple Model',
    fields=[
        {'name': 'Question'},
        {'name': 'Answer'},
    ],
    templates=[
    {
        'name': 'Card 1',
        'qfmt': '{{Question}}',
        'afmt': '{{FrontSide}}<hr id="answer">{{Answer}}',
    },
    ]
)

for i in range(len(df)):
    row = df.iloc[i]
    record = row_to_note(row)
    note = genanki.Note(model=my_model, fields=record)
    my_deck.add_note(note)

genanki.Package(my_deck).write_to_file('Collins_1.apkg')